### **Curating bcc_face_ear1.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp12887268.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 1148 × 20062
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.17480521, -0.1043229 , -0.07002215, ..., -0.05115629,
         0.6308113 , -0.10372975],
       [-0.17480521, -0.1043229 , -0.07002215, ..., -0.05115629,
        -0.4260805 , -0.10372975],
       [-0.17480521, -0.1043229 , -0.07002215, ..., -0.05115629,
        -0.4260805 , -0.10372975],
       ...,
       [-0.17480521, -0.1043229 , -0.07002215, ..., -0.05115629,
         4.006282  , -0.10372975],
       [-0.17480521, -0.1043229 , -0.07002215, ..., -0.05115629,
        -0.4260805 , -0.10372975],
       [-0.17480521, -0.1043229 , -0.07002215, ..., -0.05115629,
        -0.4260805 , -0.10372975]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<1148x20062 sparse matrix of type '<class 'numpy.float32'>'
	with 23031176 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.17480521
  (0, 1)	-0.1043229
  (0, 2)	-0.07002215
  (0, 3)	-0.07488585
  (0, 4)	-0.3028896
  (0, 5)	-0.33646542
  (0, 6)	-0.065174714
  (0, 7)	-0.20952412
  (0, 8)	-0.19064158
  (0, 9)	0.79645705
  (0, 10)	0.16194032
  (0, 11)	0.8137342
  (0, 12)	-0.30883506
  (0, 13)	-0.20746769
  (0, 14)	1.002832
  (0, 15)	0.31426883
  (0, 16)	0.5360879
  (0, 17)	1.0859916
  (0, 18)	0.13671388
  (0, 19)	-0.34740043
  (0, 20)	-0.070795946
  (0, 21)	-0.34571084
  (0, 22)	-0.3258446
  (0, 23)	-0.1612662
  (0, 24)	-0.58746046
  :	:
  (1147, 20037)	-0.68080324
  (1147, 20038)	-1.9955994
  (1147, 20039)	-1.8052659
  (1147, 20040)	-1.592896
  (1147, 20041)	-0.5784179
  (1147, 20042)	-1.1575558
  (1147, 20043)	-1.2549756
  (1147, 20044)	-0.8770976
  (1147, 20045)	0.0918837
  (1147, 20046)	-1.0096287
  (1147, 20047)	-1.7076657
  (1147, 20048)	-0.18051401
  (1147, 20049)	-0.85568386
  (1147, 20050)	-0.05878398
  (1147, 20051)	-0.13698089
  (1147, 20052)	-0.12238642
  (1147, 20053)	-0.11073064
  (11

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp12887268/filtered_feature_bc_matrix.h5')

/nfs/users/nfs_s/sa32/anaconda/envs/shibla_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 1148 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<1148x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 5257692 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 23)	2.0
  (0, 24)	3.0
  (0, 25)	2.0
  (0, 29)	10.0
  (0, 30)	4.0
  (0, 32)	3.0
  (0, 35)	1.0
  (0, 36)	1.0
  (0, 44)	6.0
  (0, 48)	10.0
  (0, 50)	1.0
  (0, 52)	1.0
  (0, 53)	6.0
  (0, 55)	1.0
  (0, 57)	3.0
  (0, 58)	16.0
  (0, 59)	26.0
  (0, 60)	9.0
  (0, 61)	1.0
  (0, 62)	12.0
  (0, 68)	2.0
  (0, 70)	1.0
  (0, 72)	1.0
  (0, 73)	13.0
  (0, 78)	6.0
  :	:
  (1147, 36389)	1.0
  (1147, 36398)	5.0
  (1147, 36400)	1.0
  (1147, 36401)	36.0
  (1147, 36404)	1.0
  (1147, 36406)	1.0
  (1147, 36407)	3.0
  (1147, 36409)	1.0
  (1147, 36412)	1.0
  (1147, 36415)	1.0
  (1147, 36435)	1.0
  (1147, 36445)	1.0
  (1147, 36450)	2.0
  (1147, 36470)	1.0
  (1147, 36516)	1.0
  (1147, 36559)	20.0
  (1147, 36560)	2.0
  (1147, 36561)	9.0
  (1147, 36562)	29.0
  (1147, 36564)	13.0
  (1147, 36565)	24.0
  (1147, 36566)	14.0
  (1147, 36567)	1.0
  (1147, 36568)	15.0
  (1147, 36571)	13.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
AL627309.5
AP006222.2
AL732372.1
LINC01409
...
AC004556.3
AC233755.1
AC141272.1
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
AL627309.5,AL627309.5,ENSG00000241860
AP006222.2,AP006222.2,ENSG00000286448
AL732372.1,AL732372.1,ENSG00000236601
LINC01409,LINC01409,ENSG00000237491
...,...,...
AC004556.3,AC004556.3,ENSG00000276345
AC233755.1,AC233755.1,ENSG00000275063
AC141272.1,AC141272.1,ENSG00000277836
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

18

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000241860,AL627309.5,ENSG00000241860
ENSG00000286448,AP006222.2,ENSG00000286448
ENSG00000236601,AL732372.1,ENSG00000236601
ENSG00000237491,LINC01409,ENSG00000237491
...,...,...
ENSG00000276345,AC004556.3,ENSG00000276345
ENSG00000275063,AC233755.1,ENSG00000275063
ENSG00000277836,AC141272.1,ENSG00000277836


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000286448
ENSG00000236601
ENSG00000237491
...
ENSG00000276345
ENSG00000275063
ENSG00000277836


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/Endometrium_reference_integrated_atlas/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

119799

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

19959

In [57]:
len(var_to_keep_araw)

36390

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000286448
ENSG00000236601
ENSG00000237491
...
ENSG00000276345
ENSG00000275063
ENSG00000277836


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,False
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

16431

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,0.003449,0.036540,0.025108,0.002104,0.000476,8,0.771441,0.791477,0.000028,0.000461
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,0.016863,0.010076,0.015221,0.005964,0.000687,1,0.339927,0.009676,0.005392,0.010503
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,0.011660,0.000082,0.143640,0.000016,0.102151,4,2.012764,1.591530,0.000426,0.000274
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,0.017567,0.020014,0.254544,0.162443,0.026544,2,0.802377,0.015617,0.272136,0.196184
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,0.026274,0.052605,0.003052,0.000043,0.005599,6,0.436219,0.805431,0.000025,0.004832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,0.043526,0.007594,0.170871,0.106217,0.047389,0,1.367711,0.073417,0.065311,0.024647
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,0.042123,0.004540,0.281694,0.262012,0.016993,2,0.673912,0.009343,0.087371,0.105637
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,0.093782,0.027256,0.072545,0.055619,0.235628,2,1.907076,0.079030,0.416707,0.072896


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,0.036540,0.025108,0.002104,0.000476,8,0.771441,0.791477,0.000028,0.000461,EFO:0010961
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,0.010076,0.015221,0.005964,0.000687,1,0.339927,0.009676,0.005392,0.010503,EFO:0010961
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,0.000082,0.143640,0.000016,0.102151,4,2.012764,1.591530,0.000426,0.000274,EFO:0010961
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,0.020014,0.254544,0.162443,0.026544,2,0.802377,0.015617,0.272136,0.196184,EFO:0010961
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,0.052605,0.003052,0.000043,0.005599,6,0.436219,0.805431,0.000025,0.004832,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,0.007594,0.170871,0.106217,0.047389,0,1.367711,0.073417,0.065311,0.024647,EFO:0010961
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,0.004540,0.281694,0.262012,0.016993,2,0.673912,0.009343,0.087371,0.105637,EFO:0010961
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,0.027256,0.072545,0.055619,0.235628,2,1.907076,0.079030,0.416707,0.072896,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,0.002104,0.000476,8,0.771441,0.791477,0.000028,0.000461,EFO:0010961,c2l_POSTN+ fibroblasts,1.094982
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,0.005964,0.000687,1,0.339927,0.009676,0.005392,0.010503,EFO:0010961,c2l_POSTN+ fibroblasts,1.745876
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,0.000016,0.102151,4,2.012764,1.591530,0.000426,0.000274,EFO:0010961,c2l_Suprabasal keratinocytes,2.012764
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,0.162443,0.026544,2,0.802377,0.015617,0.272136,0.196184,EFO:0010961,c2l_TAGLN+ pericytes,3.901541
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,0.000043,0.005599,6,0.436219,0.805431,0.000025,0.004832,EFO:0010961,c2l_POSTN+ fibroblasts,1.605923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,0.106217,0.047389,0,1.367711,0.073417,0.065311,0.024647,EFO:0010961,c2l_PlasmaC,2.735267
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,0.262012,0.016993,2,0.673912,0.009343,0.087371,0.105637,EFO:0010961,c2l_VEC,3.691354
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,0.055619,0.235628,2,1.907076,0.079030,0.416707,0.072896,EFO:0010961,c2l_PlasmaC,2.179551


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [94]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['CL:0000057',
 'CL:4033013',
 'CL:0000669',
 'CL:0000646',
 'CL:0000786',
 'CL:0002139',
 'CL:0002187',
 'CL:0000910',
 'CL:0002573',
 'CL:0000623',
 'CL:0000235',
 'CL:0000138',
 'CL:0002138',
 'CL:0000815',
 'CL:0001203']

In [95]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,0.000476,8,0.771441,0.791477,0.000028,0.000461,EFO:0010961,c2l_POSTN+ fibroblasts,1.094982,CL:0000057
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,0.000687,1,0.339927,0.009676,0.005392,0.010503,EFO:0010961,c2l_POSTN+ fibroblasts,1.745876,CL:0000057
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,0.102151,4,2.012764,1.591530,0.000426,0.000274,EFO:0010961,c2l_Suprabasal keratinocytes,2.012764,CL:4033013
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,0.026544,2,0.802377,0.015617,0.272136,0.196184,EFO:0010961,c2l_TAGLN+ pericytes,3.901541,CL:0000669
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,0.005599,6,0.436219,0.805431,0.000025,0.004832,EFO:0010961,c2l_POSTN+ fibroblasts,1.605923,CL:0000057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,0.047389,0,1.367711,0.073417,0.065311,0.024647,EFO:0010961,c2l_PlasmaC,2.735267,CL:0000786
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,0.016993,2,0.673912,0.009343,0.087371,0.105637,EFO:0010961,c2l_VEC,3.691354,CL:0002139
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,0.235628,2,1.907076,0.079030,0.416707,0.072896,EFO:0010961,c2l_PlasmaC,2.179551,CL:0000786


In [96]:
araw.obs

""
AAACCCGAACGAAATC-1
AAACGAGACGGTTGAT-1
AAACTGCTGGCTCCAA-1
AAAGGGATGTAGCAAG-1
AAAGTGTGATTTATCT-1
...
TTGTCGTTCAGTTACC-1
TTGTGGCCCTGACAGT-1
TTGTTAGCAAATTCGA-1
TTGTTCAGTGTGCTAC-1


#### **donor_id**

In [97]:
#identify the column in adata.obs which provides donor information

In [98]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [99]:
# add the donor_id column

In [100]:
adata.obs['donor_id'] = ['bcc_face_ear_A'] * len(adata.obs)

In [101]:
# change datatype of the column

In [102]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [103]:
# view unique values of donor_id column

In [104]:
list(adata.obs['donor_id'].unique())

['bcc_face_ear_A']

In [105]:
#view obs

In [106]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,8,0.771441,0.791477,0.000028,0.000461,EFO:0010961,c2l_POSTN+ fibroblasts,1.094982,CL:0000057,bcc_face_ear_A
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,1,0.339927,0.009676,0.005392,0.010503,EFO:0010961,c2l_POSTN+ fibroblasts,1.745876,CL:0000057,bcc_face_ear_A
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,4,2.012764,1.591530,0.000426,0.000274,EFO:0010961,c2l_Suprabasal keratinocytes,2.012764,CL:4033013,bcc_face_ear_A
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,2,0.802377,0.015617,0.272136,0.196184,EFO:0010961,c2l_TAGLN+ pericytes,3.901541,CL:0000669,bcc_face_ear_A
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,6,0.436219,0.805431,0.000025,0.004832,EFO:0010961,c2l_POSTN+ fibroblasts,1.605923,CL:0000057,bcc_face_ear_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,0,1.367711,0.073417,0.065311,0.024647,EFO:0010961,c2l_PlasmaC,2.735267,CL:0000786,bcc_face_ear_A
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,2,0.673912,0.009343,0.087371,0.105637,EFO:0010961,c2l_VEC,3.691354,CL:0002139,bcc_face_ear_A
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,2,1.907076,0.079030,0.416707,0.072896,EFO:0010961,c2l_PlasmaC,2.179551,CL:0000786,bcc_face_ear_A


In [107]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [108]:
# identify the column in adata which corresponds to age

In [109]:
# add the development_stage_ontology_term_id column

In [110]:
adata.obs['development_stage_ontology_term_id'] = ['HsapDv:0000216'] * len(adata.obs)

In [111]:
# change datatype of the column

In [112]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [113]:
# view unique values of development_stage_ontology_term_id column

In [114]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000216']

In [115]:
# view adata.obs

In [116]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,0.771441,0.791477,0.000028,0.000461,EFO:0010961,c2l_POSTN+ fibroblasts,1.094982,CL:0000057,bcc_face_ear_A,HsapDv:0000216
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,0.339927,0.009676,0.005392,0.010503,EFO:0010961,c2l_POSTN+ fibroblasts,1.745876,CL:0000057,bcc_face_ear_A,HsapDv:0000216
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,2.012764,1.591530,0.000426,0.000274,EFO:0010961,c2l_Suprabasal keratinocytes,2.012764,CL:4033013,bcc_face_ear_A,HsapDv:0000216
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,0.802377,0.015617,0.272136,0.196184,EFO:0010961,c2l_TAGLN+ pericytes,3.901541,CL:0000669,bcc_face_ear_A,HsapDv:0000216
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,0.436219,0.805431,0.000025,0.004832,EFO:0010961,c2l_POSTN+ fibroblasts,1.605923,CL:0000057,bcc_face_ear_A,HsapDv:0000216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,1.367711,0.073417,0.065311,0.024647,EFO:0010961,c2l_PlasmaC,2.735267,CL:0000786,bcc_face_ear_A,HsapDv:0000216
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,0.673912,0.009343,0.087371,0.105637,EFO:0010961,c2l_VEC,3.691354,CL:0002139,bcc_face_ear_A,HsapDv:0000216
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,1.907076,0.079030,0.416707,0.072896,EFO:0010961,c2l_PlasmaC,2.179551,CL:0000786,bcc_face_ear_A,HsapDv:0000216


#### **disease_ontology_term_id**

In [117]:
# Assign normal since all are healthy patients

In [118]:
# add the disease_ontology_term_id column

In [119]:
adata.obs['disease_ontology_term_id'] = ['MONDO:0020804'] * len(adata.obs)

In [120]:
#change data type of column

In [121]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [122]:
# view obs

In [123]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,0.791477,0.000028,0.000461,EFO:0010961,c2l_POSTN+ fibroblasts,1.094982,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,0.009676,0.005392,0.010503,EFO:0010961,c2l_POSTN+ fibroblasts,1.745876,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,1.591530,0.000426,0.000274,EFO:0010961,c2l_Suprabasal keratinocytes,2.012764,CL:4033013,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,0.015617,0.272136,0.196184,EFO:0010961,c2l_TAGLN+ pericytes,3.901541,CL:0000669,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,0.805431,0.000025,0.004832,EFO:0010961,c2l_POSTN+ fibroblasts,1.605923,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,0.073417,0.065311,0.024647,EFO:0010961,c2l_PlasmaC,2.735267,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,0.009343,0.087371,0.105637,EFO:0010961,c2l_VEC,3.691354,CL:0002139,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,0.079030,0.416707,0.072896,EFO:0010961,c2l_PlasmaC,2.179551,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804


#### **is_primary_data**

In [124]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [125]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,0.000028,0.000461,EFO:0010961,c2l_POSTN+ fibroblasts,1.094982,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,0.005392,0.010503,EFO:0010961,c2l_POSTN+ fibroblasts,1.745876,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,0.000426,0.000274,EFO:0010961,c2l_Suprabasal keratinocytes,2.012764,CL:4033013,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,0.272136,0.196184,EFO:0010961,c2l_TAGLN+ pericytes,3.901541,CL:0000669,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,0.000025,0.004832,EFO:0010961,c2l_POSTN+ fibroblasts,1.605923,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,0.065311,0.024647,EFO:0010961,c2l_PlasmaC,2.735267,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,0.087371,0.105637,EFO:0010961,c2l_VEC,3.691354,CL:0002139,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,0.416707,0.072896,EFO:0010961,c2l_PlasmaC,2.179551,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True


In [126]:
#change data type of column

In [127]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [128]:
# assign organism id 

In [129]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [130]:
#change data type of column

In [131]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [132]:
# view obs

In [133]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,0.000461,EFO:0010961,c2l_POSTN+ fibroblasts,1.094982,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,0.010503,EFO:0010961,c2l_POSTN+ fibroblasts,1.745876,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,0.000274,EFO:0010961,c2l_Suprabasal keratinocytes,2.012764,CL:4033013,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,0.196184,EFO:0010961,c2l_TAGLN+ pericytes,3.901541,CL:0000669,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,0.004832,EFO:0010961,c2l_POSTN+ fibroblasts,1.605923,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,0.024647,EFO:0010961,c2l_PlasmaC,2.735267,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,0.105637,EFO:0010961,c2l_VEC,3.691354,CL:0002139,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,0.072896,EFO:0010961,c2l_PlasmaC,2.179551,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [134]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [135]:
# change data type

In [136]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [137]:
# view obs

In [138]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,EFO:0010961,c2l_POSTN+ fibroblasts,1.094982,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,EFO:0010961,c2l_POSTN+ fibroblasts,1.745876,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,2.012764,CL:4033013,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,EFO:0010961,c2l_TAGLN+ pericytes,3.901541,CL:0000669,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,EFO:0010961,c2l_POSTN+ fibroblasts,1.605923,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,EFO:0010961,c2l_PlasmaC,2.735267,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,EFO:0010961,c2l_VEC,3.691354,CL:0002139,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,EFO:0010961,c2l_PlasmaC,2.179551,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown


In [139]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [140]:
adata.obs['sex_ontology_term_id'] = ['PATO:0000384'] * len(adata.obs)

In [141]:
# change data type

In [142]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [143]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,c2l_POSTN+ fibroblasts,1.094982,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,c2l_POSTN+ fibroblasts,1.745876,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,c2l_Suprabasal keratinocytes,2.012764,CL:4033013,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,c2l_TAGLN+ pericytes,3.901541,CL:0000669,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,c2l_POSTN+ fibroblasts,1.605923,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,c2l_PlasmaC,2.735267,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,c2l_VEC,3.691354,CL:0002139,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,c2l_PlasmaC,2.179551,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384


#### **suspension_type**

In [144]:
# since visium suspension type is 'na'

In [145]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [146]:
# change data type

In [147]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [148]:
# view obs

In [149]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,1.094982,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,1.745876,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,2.012764,CL:4033013,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,3.901541,CL:0000669,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,1.605923,CL:0000057,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,2.735267,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,3.691354,CL:0002139,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,2.179551,CL:0000786,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na


#### **tissue_type**

In [150]:
adata.obs['tissue_type'] = ['tissue'] * len(adata.obs)

In [151]:
adata.obs['tissue_type'] = adata.obs['tissue_type'].astype('category')

#### **tissue_ontology_term_id**

In [152]:
adata.obs['tissue_ontology_term_id'] = ['UBERON:0001459'] * len(adata.obs)

In [153]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [154]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:0001459']

In [155]:
# view obs

In [156]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,45,115,10171,75286,1506,2.000372,7490,9.948729,AAACCCGAACGAAATC-1,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAACGAGACGGTTGAT-1,1,35,79,5081,18322,193,1.053378,2153,11.750900,AAACGAGACGGTTGAT-1,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAACTGCTGGCTCCAA-1,1,45,67,10340,77226,2055,2.661021,7062,9.144588,AAACTGCTGGCTCCAA-1,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAAGGGATGTAGCAAG-1,1,24,62,3837,17849,504,2.823688,1877,10.515995,AAAGGGATGTAGCAAG-1,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAAGTGTGATTTATCT-1,1,44,94,8633,42452,1036,2.440403,4840,11.401112,AAAGTGTGATTTATCT-1,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,22,58,1526,5272,128,2.427921,570,10.811836,TTGTCGTTCAGTTACC-1,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
TTGTGGCCCTGACAGT-1,1,18,60,2680,8298,267,3.217643,1041,12.545193,TTGTGGCCCTGACAGT-1,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
TTGTTAGCAAATTCGA-1,1,22,42,1960,5931,264,4.451189,897,15.123924,TTGTTAGCAAATTCGA-1,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459


In [157]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [158]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [159]:
adata.obsm

AxisArrays with keys: X_spatial

In [160]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [161]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([777.9398  , 237.05823 , 137.31102 , 117.73552 ,  71.753265,
        63.537872,  58.74141 ,  44.752663,  41.74479 ,  41.001606,
        39.82293 ,  38.756844,  37.604977,  37.205322,  36.929546,
        36.67021 ,  36.20711 ,  36.126286,  35.88258 ,  35.75908 ,
        35.38896 ,  35.32545 ,  34.9724  ,  34.861782,  34.805954,
        34.569435,  34.469677,  34.294872,  34.0359  ,  33.893642,
        33.813328,  33.57087 ,  33.52807 ,  33.457558,  33.313812,
        33.1382  ,  33.104847,  32.972084,  32.770523,  32.701626,
        32.482876,  32.35948 ,  32.2416  ,  32.146145,  32.10392 ,
        32.01063 

In [162]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the ear stained with H&E'

In [163]:
adata.uns['title'] = 'Visium spatial - bcc_face_ear_A'

In [164]:
adata.uns['default_embedding'] = 'X_spatial'

In [165]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [166]:
adata

AnnData object with n_obs × n_vars = 1148 × 36390
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_t

In [167]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [168]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [169]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [170]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,10171,75286,1506,2.000372,7490,9.948729,0.003039,0.041467,0.000053,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAACGAGACGGTTGAT-1,1,5081,18322,193,1.053378,2153,11.750900,0.025673,0.063881,0.238397,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAACTGCTGGCTCCAA-1,1,10340,77226,2055,2.661021,7062,9.144588,0.002557,0.002296,0.000126,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAAGGGATGTAGCAAG-1,1,3837,17849,504,2.823688,1877,10.515995,0.020323,0.120877,0.642590,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAAGTGTGATTTATCT-1,1,8633,42452,1036,2.440403,4840,11.401112,0.002767,0.082524,0.000582,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,1526,5272,128,2.427921,570,10.811836,0.025578,0.139907,0.966367,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
TTGTGGCCCTGACAGT-1,1,2680,8298,267,3.217643,1041,12.545193,0.036088,0.054379,1.222507,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
TTGTTAGCAAATTCGA-1,1,1960,5931,264,4.451189,897,15.123924,0.034934,0.276043,1.447813,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459


In [171]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [172]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,False
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [173]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCGAACGAAATC-1,1,10171,75286,1506,2.000372,7490,9.948729,0.003039,0.041467,0.000053,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAACGAGACGGTTGAT-1,1,5081,18322,193,1.053378,2153,11.750900,0.025673,0.063881,0.238397,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAACTGCTGGCTCCAA-1,1,10340,77226,2055,2.661021,7062,9.144588,0.002557,0.002296,0.000126,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAAGGGATGTAGCAAG-1,1,3837,17849,504,2.823688,1877,10.515995,0.020323,0.120877,0.642590,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
AAAGTGTGATTTATCT-1,1,8633,42452,1036,2.440403,4840,11.401112,0.002767,0.082524,0.000582,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTCGTTCAGTTACC-1,1,1526,5272,128,2.427921,570,10.811836,0.025578,0.139907,0.966367,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
TTGTGGCCCTGACAGT-1,1,2680,8298,267,3.217643,1041,12.545193,0.036088,0.054379,1.222507,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459
TTGTTAGCAAATTCGA-1,1,1960,5931,264,4.451189,897,15.123924,0.034934,0.276043,1.447813,...,bcc_face_ear_A,HsapDv:0000216,MONDO:0020804,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001459


In [174]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [175]:
#check the format of expression matrix

In [176]:
adata.X

<1148x36390 sparse matrix of type '<class 'numpy.float32'>'
	with 22912932 stored elements in Compressed Sparse Row format>

In [177]:
araw.X

<1148x36390 sparse matrix of type '<class 'numpy.float32'>'
	with 5251262 stored elements in Compressed Sparse Row format>

In [178]:
#Copy raw counts to adata.raw

In [179]:
adata.raw = araw

In [180]:
del adata.uns['log1p']

In [181]:
#write the curated object to final_objects folder

In [182]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/bcc_face_ear1.h5ad', compression = 'gzip')